In [1]:
import pickle
from pathlib import Path

import pyvista as pv

from cardiac_electrophysiology import mesh_processing as mp
from cardiac_electrophysiology.ulac import segmentation as seg

In [2]:
patient_id = "01"
input_mesh_file = Path(f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk")
segmentation_file_name = Path(f"../data/processed/patient_{patient_id}/segmentation.pkl")
anatomical_tags = mp.FeatureTags(MV=0, LAA=1, LIPV=2, LSPV=3, RIPV=4, RSPV=5)

mesh = pv.read(input_mesh_file)
mesh = mp.convert_unstructured_to_polydata_mesh(mesh)

In [3]:
boundary_paths = seg.get_boundary_paths(mesh, anatomical_tags)
connection_paths = seg.construct_connection_paths(mesh, boundary_paths)
markers = seg.get_markers(connection_paths)
parameterized_paths = seg.parameterize_paths(mesh, boundary_paths, connection_paths, markers)

objects_to_save = [boundary_paths, connection_paths, markers, parameterized_paths]
segmentation_file_name.parent.mkdir(parents=True, exist_ok=True)
with segmentation_file_name.open("wb") as f:
    pickle.dump(objects_to_save, f)

In [4]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(
    mesh,
    color="lightgrey",
    show_edges=True,
    edge_opacity=0.3,
)
for path_field in ["LIPV", "LSPV", "RIPV", "RSPV"]:
    for boundary_type in ["inner", "outer"]:
        path = getattr(parameterized_paths, f"{path_field}_{boundary_type}")
        path_mesh = pv.PolyData(mesh.points[path.inds])
        path_mesh.point_data["relative_lengths"] = path.relative_lengths
        plotter.add_points(
            path_mesh, scalars="relative_lengths", point_size=5, render_points_as_spheres=True
        )

path = parameterized_paths.LAA
path_mesh = pv.PolyData(mesh.points[path.inds])
path_mesh.point_data["relative_lengths"] = path.relative_lengths
plotter.add_points(
    path_mesh, scalars="relative_lengths", point_size=5, render_points_as_spheres=True
)
path = parameterized_paths.MV
path_mesh = pv.PolyData(mesh.points[path.inds])
path_mesh.point_data["relative_lengths"] = path.relative_lengths
plotter.add_points(
    path_mesh, scalars="relative_lengths", point_size=5, render_points_as_spheres=True
)

plotter.show()

Widget(value='<iframe src="http://localhost:33213/index.html?ui=P_0x7f5c24566e40_0&reconnect=auto" class="pyvi…